# Keltner Strategy Analysis

Analyzing the latest backtest run with sparse signal traces.

In [ ]:
# Setup
import sys
sys.path.append('../../src')  # Add src to path

import duckdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
pd.options.display.float_format = '{:.4f}'.format

In [ ]:
# Load data using our simple analytics
from analytics import TraceAnalysis

# Point to the latest run
ta = TraceAnalysis('results/20250622_155944')
print(ta)

In [ ]:
# Quick overview - what do we have?
overview = ta.sql("""
    SELECT 
        COUNT(DISTINCT strategy_id) as num_strategies,
        COUNT(*) as total_signals,
        MIN(idx) as first_bar,
        MAX(idx) as last_bar,
        COUNT(DISTINCT idx) as unique_bars
    FROM traces
""")
overview

In [ ]:
# Load metadata to understand strategies
with open('results/20250622_155944/metadata.json') as f:
    metadata = json.load(f)

print(f"Total strategies: {len([k for k in metadata['components'] if 'strategy' in k])}")
print(f"Total bars: {metadata['total_bars']}")
print(f"Total signals: {metadata['total_signals']}")
print(f"Compression ratio: {metadata['compression_ratio']:.2f}")

## Signal Activity Analysis

In [ ]:
# Analyze signal activity by strategy
signal_activity = ta.sql("""
    SELECT 
        strategy_id,
        COUNT(*) as num_signals,
        SUM(CASE WHEN val > 0 THEN 1 ELSE 0 END) as long_signals,
        SUM(CASE WHEN val < 0 THEN 1 ELSE 0 END) as short_signals,
        AVG(ABS(val)) as avg_signal_strength,
        MIN(px) as min_price,
        MAX(px) as max_price
    FROM traces
    GROUP BY strategy_id
    ORDER BY num_signals DESC
""")

print(f"Strategies analyzed: {len(signal_activity)}")
signal_activity.head(10)

In [ ]:
# Distribution of signal counts
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
signal_activity['num_signals'].hist(bins=50, alpha=0.7)
plt.xlabel('Number of Signals')
plt.ylabel('Strategy Count')
plt.title('Distribution of Signal Counts')

plt.subplot(1, 2, 2)
signal_activity['long_signals'].plot.scatter(x=signal_activity.index, y='long_signals', alpha=0.5, label='Long')
signal_activity['short_signals'].plot.scatter(x=signal_activity.index, y='short_signals', alpha=0.5, label='Short', ax=plt.gca())
plt.xlabel('Strategy ID')
plt.ylabel('Signal Count')
plt.title('Long vs Short Signals by Strategy')
plt.legend()

plt.tight_layout()
plt.show()

## Trade Analysis

Let's extract trades from the sparse signals. A trade occurs when:
- Signal goes from 0 to non-zero (entry)
- Signal goes from non-zero to 0 (exit)
- Signal reverses (exit and new entry)

In [ ]:
# Extract trades using SQL window functions
trades = ta.sql("""
    WITH signal_changes AS (
        SELECT 
            strategy_id,
            idx as bar_idx,
            val as signal_value,
            px as price,
            LAG(val, 1, 0) OVER (PARTITION BY strategy_id ORDER BY idx) as prev_signal,
            LEAD(val, 1, 0) OVER (PARTITION BY strategy_id ORDER BY idx) as next_signal,
            LEAD(idx, 1) OVER (PARTITION BY strategy_id ORDER BY idx) as next_bar,
            LEAD(px, 1) OVER (PARTITION BY strategy_id ORDER BY idx) as next_price
        FROM traces
    ),
    entries AS (
        SELECT 
            strategy_id,
            bar_idx as entry_bar,
            price as entry_price,
            signal_value as entry_signal,
            ROW_NUMBER() OVER (PARTITION BY strategy_id ORDER BY bar_idx) as entry_num
        FROM signal_changes
        WHERE signal_value != 0 AND prev_signal = 0
    ),
    exits AS (
        SELECT 
            strategy_id,
            bar_idx as exit_bar,
            price as exit_price,
            ROW_NUMBER() OVER (PARTITION BY strategy_id ORDER BY bar_idx) as exit_num
        FROM signal_changes
        WHERE signal_value = 0 AND prev_signal != 0
    )
    SELECT 
        e.strategy_id,
        e.entry_bar,
        e.entry_price,
        e.entry_signal,
        x.exit_bar,
        x.exit_price,
        (x.exit_bar - e.entry_bar) as duration_bars,
        CASE 
            WHEN e.entry_signal > 0 THEN 'long'
            ELSE 'short'
        END as direction,
        CASE 
            WHEN e.entry_signal > 0 THEN (x.exit_price - e.entry_price) / e.entry_price
            ELSE (e.entry_price - x.exit_price) / e.entry_price
        END as return_pct
    FROM entries e
    LEFT JOIN exits x 
        ON e.strategy_id = x.strategy_id 
        AND x.exit_num = e.entry_num
    WHERE x.exit_bar IS NOT NULL
    ORDER BY e.strategy_id, e.entry_bar
""")

print(f"Total trades extracted: {len(trades)}")
trades.head()

In [ ]:
# Performance summary by strategy
performance = trades.groupby('strategy_id').agg({
    'return_pct': ['count', 'mean', 'std', 'min', 'max'],
    'duration_bars': 'mean'
}).round(4)

performance.columns = ['num_trades', 'avg_return', 'return_std', 'worst_trade', 'best_trade', 'avg_duration']
performance['sharpe'] = performance['avg_return'] / performance['return_std'] * np.sqrt(252) # Annualized
performance['win_rate'] = trades[trades['return_pct'] > 0].groupby('strategy_id').size() / performance['num_trades']

# Top performers
top_performers = performance.sort_values('sharpe', ascending=False).head(20)
top_performers

In [ ]:
# Visualize performance distribution
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Sharpe distribution
axes[0, 0].hist(performance['sharpe'].dropna(), bins=50, alpha=0.7, color='blue')
axes[0, 0].axvline(performance['sharpe'].mean(), color='red', linestyle='--', label=f'Mean: {performance["sharpe"].mean():.2f}')
axes[0, 0].set_xlabel('Sharpe Ratio')
axes[0, 0].set_ylabel('Count')
axes[0, 0].set_title('Sharpe Ratio Distribution')
axes[0, 0].legend()

# Win rate distribution
axes[0, 1].hist(performance['win_rate'].dropna(), bins=50, alpha=0.7, color='green')
axes[0, 1].axvline(0.5, color='red', linestyle='--', label='50% Win Rate')
axes[0, 1].set_xlabel('Win Rate')
axes[0, 1].set_ylabel('Count')
axes[0, 1].set_title('Win Rate Distribution')
axes[0, 1].legend()

# Average return vs risk
scatter = axes[1, 0].scatter(performance['return_std'], performance['avg_return'], 
                            c=performance['sharpe'], cmap='RdYlGn', alpha=0.6, s=50)
axes[1, 0].set_xlabel('Return Std Dev')
axes[1, 0].set_ylabel('Average Return')
axes[1, 0].set_title('Risk vs Return')
plt.colorbar(scatter, ax=axes[1, 0], label='Sharpe')

# Number of trades vs Sharpe
axes[1, 1].scatter(performance['num_trades'], performance['sharpe'], alpha=0.6)
axes[1, 1].set_xlabel('Number of Trades')
axes[1, 1].set_ylabel('Sharpe Ratio')
axes[1, 1].set_title('Trade Frequency vs Performance')

plt.tight_layout()
plt.show()

## Parameter Analysis

Since we ran many Keltner strategies with different parameters, let's see which parameters work best.

In [ ]:
# Extract parameters from metadata
strategy_params = []
for comp_id, comp_data in metadata['components'].items():
    if 'strategy' in comp_id:
        # Extract strategy ID from component name
        strategy_id = int(comp_id.split('_')[-1])
        
        # For now, we'll use placeholder params since they're compiled
        # In a real scenario, you'd have the actual period/multiplier values
        strategy_params.append({
            'strategy_id': strategy_id,
            'signal_changes': comp_data['signal_changes'],
            'compression_ratio': comp_data['compression_ratio']
        })

params_df = pd.DataFrame(strategy_params)

# Merge with performance
analysis_df = params_df.merge(performance, on='strategy_id', how='inner')
print(f"Strategies with both params and performance: {len(analysis_df)}")
analysis_df.head()

In [ ]:
# Analyze relationship between signal frequency and performance
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(analysis_df['signal_changes'], analysis_df['sharpe'], alpha=0.6)
plt.xlabel('Number of Signal Changes')
plt.ylabel('Sharpe Ratio')
plt.title('Signal Frequency vs Performance')

plt.subplot(1, 2, 2)
plt.scatter(analysis_df['compression_ratio'], analysis_df['sharpe'], alpha=0.6)
plt.xlabel('Compression Ratio')
plt.ylabel('Sharpe Ratio')
plt.title('Signal Sparsity vs Performance')

plt.tight_layout()
plt.show()

## Filter Analysis

Let's check if any strategies used filters and how they performed.

In [ ]:
# Group strategies by signal count ranges to identify potential filter effects
signal_ranges = pd.cut(signal_activity['num_signals'], 
                      bins=[0, 50, 100, 150, 200, 300, 500, 1000],
                      labels=['<50', '50-100', '100-150', '150-200', '200-300', '300-500', '>500'])

signal_groups = signal_activity.groupby(signal_ranges).agg({
    'strategy_id': 'count',
    'num_signals': 'mean',
    'long_signals': 'mean',
    'short_signals': 'mean'
}).round(0)

signal_groups.columns = ['num_strategies', 'avg_signals', 'avg_long', 'avg_short']
signal_groups

## Best Strategies for Production

Let's identify the best strategies based on multiple criteria.

In [ ]:
# Filter for production-ready strategies
production_criteria = (
    (performance['sharpe'] > 1.5) &
    (performance['num_trades'] > 50) &
    (performance['win_rate'] > 0.45) &
    (performance['worst_trade'] > -0.05)  # Max 5% loss per trade
)

production_ready = performance[production_criteria].sort_values('sharpe', ascending=False)
print(f"Production-ready strategies: {len(production_ready)}")
production_ready.head(10)

In [ ]:
# Export best strategies
if len(production_ready) > 0:
    # Get the top 5 strategies
    best_strategies = production_ready.head(5)
    
    # Save to JSON for easy loading
    export_data = {
        'analysis_date': pd.Timestamp.now().isoformat(),
        'criteria': {
            'min_sharpe': 1.5,
            'min_trades': 50,
            'min_win_rate': 0.45,
            'max_worst_trade': -0.05
        },
        'strategies': best_strategies.reset_index().to_dict('records')
    }
    
    with open('production_strategies.json', 'w') as f:
        json.dump(export_data, f, indent=2)
    
    print("Saved production strategies to production_strategies.json")
else:
    print("No strategies met the production criteria")

## Custom Analysis Examples

Here are some more SQL queries you can run:

In [ ]:
# Example 1: Find strategies with consistent signals
consistent = ta.sql("""
    WITH signal_gaps AS (
        SELECT 
            strategy_id,
            idx - LAG(idx) OVER (PARTITION BY strategy_id ORDER BY idx) as gap
        FROM traces
    )
    SELECT 
        strategy_id,
        AVG(gap) as avg_gap,
        STDDEV(gap) as gap_std,
        MAX(gap) as max_gap
    FROM signal_gaps
    WHERE gap IS NOT NULL
    GROUP BY strategy_id
    HAVING STDDEV(gap) < 100
    ORDER BY avg_gap
    LIMIT 10
""")

print("Most consistent signal generators:")
consistent

In [ ]:
# Example 2: Price levels where most signals occur
price_levels = ta.sql("""
    SELECT 
        ROUND(px / 10) * 10 as price_bucket,
        COUNT(*) as signal_count,
        SUM(CASE WHEN val > 0 THEN 1 ELSE 0 END) as long_signals,
        SUM(CASE WHEN val < 0 THEN 1 ELSE 0 END) as short_signals,
        COUNT(DISTINCT strategy_id) as active_strategies
    FROM traces
    GROUP BY price_bucket
    ORDER BY signal_count DESC
""")

# Plot price level activity
plt.figure(figsize=(10, 6))
plt.bar(price_levels['price_bucket'], price_levels['signal_count'], width=8, alpha=0.7)
plt.xlabel('Price Level')
plt.ylabel('Signal Count')
plt.title('Signal Activity by Price Level')
plt.show()

## Summary

This notebook demonstrates how to:
1. Load sparse signal traces using DuckDB
2. Extract trades from signal changes
3. Calculate performance metrics
4. Identify production-ready strategies
5. Run custom SQL analysis

The key is that everything is just SQL queries on the parquet files!